# ItaData2024

## URTI

propositional analysis

In [27]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Settings

In [28]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

features = :catch9
# features = :minmax
# features = :custom

loadset = false
# loadset = true

scale = :semitones
# scale = :mel_htk

usemfcc = false
# usemfcc = true
usef0 = false
# usef0 = true

sr = 8000
audioparams = (
    usemfcc = usemfcc,
    usef0 = usef0,
    sr = sr,
    nfft = 512,
    scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    nbands = scale == :semitones ? 14 : 26,
    ncoeffs = scale == :semitones ? 7 : 13,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = usemfcc ? false : true,
)

memguard = false;
# memguard = true;
n_elems = 15;

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

@assert experiment in avail_exp "Unknown type of experiment: $experiment."

findhealthy = y -> findall(x -> x == "Healthy", y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
findsick = y -> findall(x -> x == String(experiment), y)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"
memguard && begin filename *= string("_memguard") end

destpath = "results/propositional"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

### Audio features extraction function
This function is called for every audio sample and extract 25 features:
14 bands of the mel spectrogram,
11 spectral features: centroid, crest, entropy, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [29]:
function afe(x::AbstractVector{Float64}, audioparams::NamedTuple; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = audioparams.sr
    norm = true
    speech_detection = false
    # stft module
    nfft = audioparams.nfft
    win_type = (:hann, :periodic)
    win_length = audioparams.nfft
    overlap_length = round(Int, audioparams.nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = audioparams.nbands
    scale = audioparams.scale               # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = audioparams.freq_range
    # mel spectrogram module
    db_scale = audioparams.db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    if audioparams.usemfcc
        # mfcc module
        ncoeffs = audioparams.ncoeffs
        rectification = :log                    # :log, :cubic_root
        dither = true

        mfcc = get_mfcc(
            source=melspec,
            ncoeffs=ncoeffs,
            rectification=rectification,
            dither=dither,
        );
    end

    if audioparams.usef0
        # f0 module
        method = :nfc
        f0_range = (50, 400)

        f0 = get_f0(
            source=stftspec,
            method=method,
            freq_range=f0_range
        );
    end

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        filter(!isnothing, [
            melspec.spec',
            audioparams.usemfcc ? mfcc.mfcc' : nothing,
            audioparams.usef0 ? f0.f0 : nothing,
            spect.centroid,
            spect.crest,
            spect.entropy,
            spect.flatness,
            spect.flux,
            spect.kurtosis,
            spect.rolloff,
            spect.skewness,
            spect.decrease,
            spect.slope,
            spect.spread
        ])...
    )    
end

afe (generic function with 2 methods)

### Prepare dataset for analysis

In [30]:
d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

memguard && begin
    cat2 = round(Int, length(y)/2)
    indices = [1:n_elems; cat2:cat2+n_elems-1]
    x = x[indices, :]
    y = y[indices]
end

freq = round.(Int, afe(x[1, :audio], audioparams; get_only_melfreq=true))

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.nbands],
        audioparams.usemfcc ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.ncoeffs] : String[],
        audioparams.usef0 ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)
    
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

if !loadset
    @info("Build dataset...")

    X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
    audiofeats = [afe(row[:audio], audioparams) for row in eachrow(x)]
    push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audiofeats])...)

    yc = CategoricalArray(y);

    train_ratio = 0.8

    train, test = partition(eachindex(yc), train_ratio, shuffle=true)
    X_train, y_train = X[train, :], yc[train]
    X_test, y_test = X[test, :], yc[test]

    println("Training set size: ", size(X_train), " - ", length(y_train))
    println("Test set size: ", size(X_test), " - ", length(y_test))
end

┌ Info: Build dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:42


Training set size: (342, 225) - 342
Test set size: (86, 225) - 86


### Train a model

In [31]:
if !loadset
    learned_dt_tree = begin
        Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
        model = Tree(max_depth=-1, )
        mach = machine(model, X_train, y_train)
        fit!(mach)
        fitted_params(mach).tree
    end
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


max(sprd) < 664.1
├─ min(mel8=1194Hz) < -4.028
│  ├─ 3ac(mel2=424Hz) < 0.1156
│  │  ├─ bsm(mel3=504Hz) < 17.5
│  │  │  ├─ qnt(mel8=1194Hz) < 1.949
│  │  │  │  ├─ URTI (63/63)
│  │  │  │  └─ mean(mel9=1419Hz) < -5.349
│  │  │  │     ⋮
│  │  │  │     
│  │  │  └─ Healthy (4/4)
│  │  └─ 3ac(mel10=1687Hz) < 0.0485
│  │     ├─ URTI (5/5)
│  │     └─ qnt(mel8=1194Hz) < 1.713
│  │        ├─ med(mel7=1005Hz) < -2.878
│  │        │  ⋮
│  │        │  
│  │        └─ Healthy (11/11)
│  └─ mean(mel13=2832Hz) < -5.051
│     ├─ Healthy (15/15)
│     └─ qnt(mel6=845Hz) < 1.809
│        ├─ Healthy (3/3)
│        └─ URTI (6/6)
└─ mean(mel1=357Hz) < -4.168
   ├─ mean(mel8=1194Hz) < -4.845
   │  ├─ Healthy (28/28)
   │  └─ URTI (1/1)
   └─ max(mel1=357Hz) < -0.1154
      ├─ qnt(mel5=711Hz) < 1.673
      │  ├─ Healthy (16/16)
      │  └─ max(sprd) < 909.0
      │     ├─ max(skwns) < 10.61
      │     │  ⋮
      │     │  
      │     └─ 3ac(sprd) < 0.04644
      │        ⋮
      │        
      └─ URTI (8/

### Model inspection & rule study

In [32]:
if !loadset
    sole_dt = solemodel(learned_dt_tree)
    # Make test instances flow into the model, so that test metrics can, then, be computed.
    apply!(sole_dt, X_test, y_test);
    # Save solemodel to disk
    jldsave(jld2file, true; sole_dt)
else
    @info("Load dataset...")
    d = jldopen(jld2file)
    sole_dt = d["sole_dt"]
    close(d)
end
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ max(sprd) < 664.0535057427451
├✔ min(mel8=1194Hz) < -4.028207814322176
│├✔ 3ac(mel2=424Hz) < 0.11555555555555556
││├✔ bsm(mel3=504Hz) < 17.5
│││├✔ qnt(mel8=1194Hz) < 1.948503881769832
││││├✔ URTI : (ninstances = 16, ncovered = 16, confidence = 0.81, lift = 1.0)
││││└✘ mean(mel9=1419Hz) < -5.349004118322046
││││ ├✔ Healthy : (ninstances = 2, ncovered = 2, confidence = 0.5, lift = 1.0)
││││ └✘ max(flatn) < 0.1489664724582898
││││  ├✔ min(mel4=599Hz) < -5.670229356292184
││││  │├✔ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││││  │└✘ URTI : (ninstances = 9, ncovered = 9, confidence = 1.0, lift = 1.0)
││││  └✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
│││└✘ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││└✘ 3ac(mel10=1687Hz) < 0.048500881834215165
││ ├✔ URTI : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
││ └✘ qnt(mel8=1194Hz) < 1.7126029972917245
││  ├✔ med(mel7=1005Hz) < -2.8777118272418427

### Extract rules that are at least as good as a random baseline model

In [33]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (max(sprd) < 664.0535057427451) ∧ (min(mel8=1194Hz) < -4.028207814322176) ∧ (3ac(mel2=424Hz) < 0.11555555555555556) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) < 1.948503881769832)  ↣  URTI : (ninstances = 86, ncovered = 16, coverage = 0.19, confidence = 0.81, lift = 1.75, natoms = 5)
▣ (max(sprd) < 664.0535057427451) ∧ (min(mel8=1194Hz) < -4.028207814322176) ∧ (3ac(mel2=424Hz) < 0.11555555555555556) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (¬(qnt(mel8=1194Hz) < 1.948503881769832)) ∧ (¬(mean(mel9=1419Hz) < -5.349004118322046)) ∧ (max(flatn) < 0.1489664724582898) ∧ (min(mel4=599Hz) < -5.670229356292184)  ↣  Healthy : (ninstances = 86, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.87, natoms = 8)
▣ (max(sprd) < 664.0535057427451) ∧ (min(mel8=1194Hz) < -4.028207814322176) ∧ (3ac(mel2=424Hz) < 0.11555555555555556) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (¬(qnt(mel8=1194Hz) < 1.948503881769832)) ∧ (¬(mean(mel9=1419Hz) < -5.349004118322046)) ∧ (max(flatn) < 0.1489664724582898) ∧ (¬(min(mel4=599Hz)

### Simplify rules while extracting and prettify result

In [34]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) < 1.95)  ↣  URTI : (ninstances = 86, ncovered = 16, coverage = 0.19, confidence = 0.81, lift = 1.75, natoms = 5)
▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) ≥ 1.95) ∧ (mean(mel9=1419Hz) ≥ -5.35) ∧ (max(flatn) < 0.15) ∧ (min(mel4=599Hz) < -5.67)  ↣  Healthy : (ninstances = 86, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.87, natoms = 8)
▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) ≥ 1.95) ∧ (mean(mel9=1419Hz) ≥ -5.35) ∧ (max(flatn) < 0.15) ∧ (min(mel4=599Hz) ≥ -5.67)  ↣  URTI : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 1.0, lift = 2.15, natoms = 8)
▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz)

### Directly access rule metrics

In [35]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

19-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 86, ncovered = 16, coverage = 0.18604651162790697, confidence = 0.8125, lift = 1.746875, natoms = 5)
 (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 8)
 (ninstances = 86, ncovered = 9, coverage = 0.10465116279069768, confidence = 1.0, lift = 2.15, natoms = 8)
 (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 7)
 (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 5)
 (ninstances = 86, ncovered = 3, coverage = 0.03488372093023256, confidence = 1.0, lift = 1.8695652173913044, natoms = 3)
 (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 4)
 (ninstances = 86, ncove

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [36]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (max(sprd) < 664.0535057427451) ∧ (min(mel8=1194Hz) ≥ -4.028207814322176) ∧ (mean(mel13=2832Hz) ≥ -5.051449874154914) ∧ (qnt(mel6=845Hz) < 1.8093160599313536)  ↣  Healthy : (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 4, height = 3)
▣ (max(sprd) < 664.0535057427451) ∧ (min(mel8=1194Hz) < -4.028207814322176) ∧ (3ac(mel2=424Hz) ≥ 0.11555555555555556) ∧ (3ac(mel10=1687Hz) ≥ 0.048500881834215165) ∧ (qnt(mel8=1194Hz) ≥ 1.7126029972917245)  ↣  Healthy : (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, confidence = 1.0, lift = 1.8695652173913044, natoms = 5, height = 4)
▣ (max(sprd) ∈ [664.0535057427451,908.9659831322302)) ∧ (mean(mel1=357Hz) ≥ -4.1684365386848885) ∧ (max(mel1=357Hz) < -0.11544222361904846) ∧ (qnt(mel5=711Hz) ≥ 1.6729201667253222) ∧ (max(skwns) < 10.610955524117045) ∧ (max(mel6=845Hz) ≥ -2.200798828340653)  ↣  Healthy : (ninstances = 86, ncovered = 1, coverage = 0.011627906976744186, c

### Pretty table of rules and their metrics

In [37]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬───────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                                                                               Antecedent │ Consequent │ ninstances │ ncovered │  coverage │ confidence │    lift │ natoms │ height │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Inspect features

In [38]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) < 1.95)  ↣  URTI : (ninstances = 86, ncovered = 16, coverage = 0.19, confidence = 0.81, lift = 1.75, natoms = 5)
▣ (max(sprd) < 664.05) ∧ (min(mel8=1194Hz) < -4.03) ∧ (3ac(mel2=424Hz) < 0.12) ∧ (bsm(mel3=504Hz) < 17.5) ∧ (qnt(mel8=1194Hz) ≥ 1.95) ∧ (mean(mel9=1419Hz) ≥ -5.35) ∧ (max(flatn) < 0.15) ∧ (min(mel4=599Hz) ≥ -5.67)  ↣  URTI : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 1.0, lift = 2.15, natoms = 8)
▣ (max(sprd) ≥ 908.97) ∧ (mean(mel1=357Hz) ≥ -4.17) ∧ (max(mel1=357Hz) < -0.12) ∧ (qnt(mel5=711Hz) ≥ 1.67) ∧ (3ac(sprd) ≥ 0.05)  ↣  Healthy : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.8, lift = 1.5, natoms = 5)


In [39]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

12-element Vector{Symbol}:
 Symbol("3ac(mel2=424Hz)")
 Symbol("3ac(sprd)")
 Symbol("bsm(mel3=504Hz)")
 Symbol("max(flatn)")
 Symbol("max(mel1=357Hz)")
 Symbol("max(sprd)")
 Symbol("mean(mel1=357Hz)")
 Symbol("mean(mel9=1419Hz)")
 Symbol("min(mel4=599Hz)")
 Symbol("min(mel8=1194Hz)")
 Symbol("qnt(mel5=711Hz)")
 Symbol("qnt(mel8=1194Hz)")